In [4]:
# import des librairies
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import math

# srapping
import requests 
from bs4 import BeautifulSoup as bs
import re

# clean data
import ast
import dateparser
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud

#export data
import time
import os
from datetime import date
from datetime import datetime

In [2]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\selmane.kenzari\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\selmane.kenzari\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
# fonctions
def scrapping():
    job_offers = []
    page = 0
    # Select URL
    url = f'https://candidat.pole-emploi.fr/offres/recherche?emission=31&motsCles=data&offresPartenaires=true&range=0-19&rayon=10&tri=0'
    while True:
        
        response = requests.get(url)
        
#         # Check if the URL is not recognized or does not exist
#         if response.status_code != 200:
#             break
            
        html_content = response.text
            
        with open ('page.html', 'w', encoding='utf-8') as file:
            file.write(html_content)

        with open('page.html', 'r', encoding='utf-8') as file:
            html_content = file.read()

        soup = bs(html_content, 'html.parser')

        job_elements = soup.find_all('div', class_='media-body')
        
        for job_element in job_elements:
            # Extract the data-intitule-offre value from the job summary element
            
            if job_element.find('h2') and job_element.find('h2')['data-intitule-offre']:
                data_intitule_offre = job_element.find('h2')['data-intitule-offre']
            
                # Construct the detail page URL using the data-intitule-offre value
                detail_page_url = f'https://candidat.pole-emploi.fr/offres/recherche/detail/{data_intitule_offre}'

                # Scrape job details from the detail page
                detail_page_response = requests.get(detail_page_url)
                detail_page_content = detail_page_response.text

                with open ('page.html', 'w', encoding='utf-8') as file:
                    file.write(detail_page_content)

                with open('page.html', 'r', encoding='utf-8') as file:
                    detail_page_content = file.read()

                detail_page_soup = bs(detail_page_content, 'html.parser')
                
                # Liste des variables à extraire
                variables = {
                    'jobs': ('span', 'title'),
                    'description': ('div', 'description col-sm-8 col-md-7'),
                    'loc': ('span', 'name'),
                    'datePosted': ('span', 'datePosted'),
                    'workhours': ('dd', 'workHours'),
                    'salary': ('span', 'baseSalary'),
                    'contract_type': ('dl', 'icon-group'),
                    'experience': ('span', 'experienceRequirements'),
                    'qualifications': ('span', 'qualifications'),
                    'industry': ('span', 'industry'),
                    'company': ('h3', 't4 title'),
                    'size': ('div', 'media-body'),
                    'company_description': ('p', 'italic'),
                    'diploma': ('span', 'educationRequirements')
                }

                # Boucle pour extraire les variables
                for var, (tag, class_) in variables.items():
                    try:
                        if var in ['salary', 'contract_type']:
                            if var == 'salary':
                                try:
                                    salary_raw = detail_page_soup.find(tag, itemprop=class_)
                                    min_value = salary_raw.find('span', itemprop='minValue').get('content').strip()
                                    max_value = salary_raw.find('span', itemprop='maxValue').get('content').strip()
                                    globals()[var] = f"{min_value} - {max_value}"
                                except AttributeError:
                                    salary_match = re.search(r'content="([\d.]+)" itemprop="value"', str(salary_raw))
                                    globals()[var] = salary_match.group(1) if salary_match else math.nan
                                    
                            elif var == 'contract_type':
                                contract_type_raw = detail_page_soup.find(tag, class_=class_).find('dd')
                                globals()[var] = str(contract_type_raw.contents[0]).strip()
                                
                        else:
                            globals()[var] = detail_page_soup.find(tag, itemprop=class_).text.replace('\n', '').strip()
                    except AttributeError:
                        # En cas d'erreur
                        globals()[var] = math.nan


                # Extraction des autres variables spécifiques
                try:
                    skills_raw = detail_page_soup.find_all('span', itemprop='skills')
                    skills = [skill.text.strip() for skill in skills_raw]
                except AttributeError:
                    # En cas d'erreur
                    skills_raw = ''
                    skills = math.nan
                    
                try:
                    company = detail_page_soup.find('h3', class_='t4 title').text.strip()
                except AttributeError:
                    company = math.nan
                    
                try:
                    size = detail_page_soup.find('div', class_='media-body').find('p').text.strip()
                except AttributeError:
                    size = math.nan
                    
                try:                    
                    company_description = detail_page_soup.find('p', class_='italic').text.replace('\n', '').strip()
                except AttributeError:
                    company_description = math.nan
                
                try: 
                    description = detail_page_soup.find('div', class_='description col-sm-8 col-md-7').text.strip()
                except AttributeError:
                    description = math.nan
                    

                # Create a dictionary for the scraped data
                data = {
                    'jobs': jobs,
                    'description': description,
                    'loc': loc,
                    'datePosted': datePosted,
                    'workhours': workhours,
                    'salary': salary,
                    'contract_type': contract_type,
                    'experience': experience,
                    'diploma': diploma,
                    'skills': skills,
                    'qualifications': qualifications,
                    'industry': industry,
                    'company': company,
                    'size': size,
                    'company_description': company_description
                }

                job_offers.append(data)
            
            else : 
                print(f'fin de la page {page}!')

        # Extract the href of the next page button
        next_page_link = soup.find('div', id='zoneAfficherPlus').find('a')
        if next_page_link:
            href_main_page = next_page_link['href']
        else:
            break
        
        # Construct the URL for the next page
        next_page_url = f'https://candidat.pole-emploi.fr{href_main_page}'
        
        # Delay before navigating to the next page
        time.sleep(60)
        
        # Update the URL to the next page
        url = next_page_url
        page += 1

    # Transform the dictionary into a dataframe
    df = pd.DataFrame(job_offers)
    
    return df

In [6]:
df = scrapping()

fin de la page 0!
fin de la page 1!
fin de la page 2!
fin de la page 3!
fin de la page 4!
fin de la page 5!
fin de la page 6!
fin de la page 7!
fin de la page 8!
fin de la page 9!
fin de la page 10!
fin de la page 11!
fin de la page 12!
fin de la page 13!
fin de la page 14!
fin de la page 15!
fin de la page 16!
fin de la page 17!
fin de la page 18!
fin de la page 19!
fin de la page 20!
fin de la page 21!
fin de la page 22!
fin de la page 23!
fin de la page 24!
fin de la page 25!
fin de la page 26!
fin de la page 27!
fin de la page 28!
fin de la page 29!
fin de la page 30!
fin de la page 31!
fin de la page 32!
fin de la page 33!
fin de la page 34!
fin de la page 35!
fin de la page 36!
fin de la page 37!
fin de la page 38!
fin de la page 39!
fin de la page 40!
fin de la page 41!
fin de la page 42!
fin de la page 43!
fin de la page 44!
fin de la page 45!
fin de la page 46!
fin de la page 47!
fin de la page 48!
fin de la page 49!


In [9]:
df.head()

,jobs,description,loc,datePosted,workhours,salary,contract_type,experience,diploma,skills,qualifications,industry,company,size,company_description
0,ANALYSTE APPLICATIF DATA FINANCE RH (H/F),Dans le cadre d'un important projet avec un gr...,92 - ISSY LES MOULINEAUX,Actualisé le 07 août 2023,35H Travail en journée,50000.0 - 60000.0,Contrat à durée indéterminée,2 ans,NaN,[Adapter les outils de traitement statistique ...,Cadre,Conseil en systèmes et logiciels informatiques,INFORMATIS T.S.,6 à 9 salariés,NaN
1,Ingénieur DATA (H/F),Nous recherchons : un Ingénieur DATA H/F.\n\nP...,92 - PUTEAUX,Actualisé le 07 août 2023,35H Travail en journée,NaN,Contrat à durée indéterminée,2 ans,"Bac+3, Bac+4 ou équivalents","[Application web, Concevoir une application we...",Agent de maîtrise,Conseil pour les affaires et autres conseils d...,IL CONSULTING,NaN,NaN
2,Data Integrity Analyst (H/F),ADECCO Lyon Pharma recherche un Data Integrity...,69 - ST PRIEST,Actualisé le 07 août 2023,35H Travail en journée,2311.0,Mission intérimaire - 12 Mois,Débutant accepté,NaN,[Adapter les outils de traitement statistique ...,Employé qualifié,Activités des agences de travail temporaire,ADECCO Onsite,0 salarié (n'ayant pas d'effectif au 31/12 mai...,"Premier réseau d'agences d'emploi en France, A..."
3,Technicien Data (h/f),"Adecco PONTARLIER, recherche pour l'un de ses ...",25 - PONTARLIER,Actualisé le 07 août 2023,35H Travail en journée,NaN,Contrat à durée indéterminée,Débutant accepté,NaN,"[Concevoir et gérer un projet, Concevoir un lo...",Employé qualifié,Activités des agences de travail temporaire,ADECCO,6 à 9 salariés,"Premier réseau d'agences d'emploi en France, A..."
4,Data Engineer (H/F),"Selon vos expériences, votre mission sera de :...",33 - BORDEAUX,Actualisé le 07 août 2023,38H30 Travail en journée,30000.0 - 50000.0,Contrat à durée indéterminée,3 ans,NaN,[],Cadre,Conseil en systèmes et logiciels informatiques,THELIO,20 à 49 salariés,NaN


In [8]:
df.shape

(1000, 15)

### Export df

In [12]:
def export_raw_data(df):
    csv_file_path = f"../data/bronze/raw_data.csv"
    df.to_csv(csv_file_path, index=False)

In [13]:
export_raw_data(df)

## Transform data

In [14]:
tools_list = ["AWS S3", "Azure Blob Storage", "Google Cloud Storage", "Dropbox", "Box", "OneDrive", "Salesforce", "HubSpot",
               "Slack", "Microsoft Teams", "Google Workspace", "Trello", "Jira", "Confluence", "GitHub", "GitLab", "Jenkins",
               "CircleCI", "Travis CI", "Docker", "Kubernetes", "Apache Hadoop", "Apache Spark", "Apache Kafka", "Apache Storm",
               "Elasticsearch", "Logstash", "Kibana", "Grafana", "Prometheus", "Nagios", "Splunk", "Tableau", "Power BI",
               "QlikView", "Looker", "Dataiku", "Alteryx", "Talend", "Informatica", "Matillion", "Databricks", "Snowflake",
               "Amazon Redshift", "Google BigQuery", "Microsoft Azure SQL Database", "Oracle Database", "MySQL", "PostgreSQL",
               "MongoDB", "Couchbase", "Cassandra", "Neo4j", "Apache Airflow", "Luigi", "Celery", "Apache NiFi", 
               "Talend Data Integration", "Google Cloud Dataflow", "Apache Beam", "Apache Flink", 
               "Microsoft Azure Data Factory", "IBM InfoSphere DataStage", "Apache Samza", "Apache Flume",
               "Apache Sqoop", "Microsoft Azure Databricks", "Teradata", "IBM Db2", "Apache Zeppelin", 
               "Jupyter Notebook", "Google Colab", "Anaconda", "TensorFlow", "PyTorch", "Keras",
               "scikit-learn", "XGBoost", "LightGBM", "H2O.ai", "Apache Mahout", "Microsoft Azure Machine Learning", 
               "IBM Watson", "RapidMiner", "KNIME", "DataRobot", "Amazon SageMaker", "Google Cloud AI Platform",
               "Domo", "IBM Cognos", "SAS", "Oracle Analytics Cloud", "Microsoft Power Automate",
               "Apache NiFi Registry", "Apache Atlas", "Apache Metron", "Apache Knox", "Apache Ranger",
               "Apache Superset", "Microsoft Power Apps", "Salesforce Einstein Analytics",
               "Google Data Studio", "Pentaho", "Microsoft Azure Synapse Analytics",
               "SAP Analytics Cloud", "MicroStrategy", "RStudio", "Apache Kylin", 
               "Apache HBase", "Apache Pig", "Apache Hive", "Apache Drill",
               "Cloudera Data Platform", "Hortonworks Data Platform", "Qubole", "DataRobot", "DataRobot Insights",
               "DataRobot MLOps", "DataRobot Paxata", "DataRobot AutoML", "DataRobot Time Series", "DataRobot AI Catalog"]

In [68]:
def create_tools(df, liste):
    tools_list = [tool.lower() for tool in liste]
    
    def find_tools(description):
        # Convertir la description en minuscules
        description = description.lower()

        # Créer une liste pour stocker les outils trouvés
        tools_found = []

        # Chercher chaque outil dans la description
        for tool in tools_list:
            if tool in description:
                tools_found.append(tool)
        
        # Si aucun outil n'a été trouvé, retourner NaN
        if not tools_found:
            return math.nan

        return ', '.join(tools_found)  # Retourner la liste d'outils trouvés
    
    df['tools'] = df['description'].apply(find_tools)
    return df

In [76]:
def new_skills(df):
    
    def clean_skills(list_skills_str):
        
        # Convertir la chaîne en une liste
        try:
            list_skills = ast.literal_eval(list_skills_str)
        except ValueError:
            # Si la conversion échoue, retourner la valeur telle quelle
            return list_skills_str
        
        # Assurez-vous que list_skills est une liste
        if isinstance(list_skills, list):
            clean_list = [skill.lower() for skill in list_skills]  # Convertir chaque compétence en minuscules
        else:
            # Si ce n'est pas une liste, retournez la valeur telle quelle
            return list_skills_str
            
        return ', '.join(clean_list)
    
    df['skills'] = df['skills'].apply(clean_skills)
    
    return df

In [79]:
df = create_tools(df, tools_list)

In [80]:
df = new_skills(df)

In [81]:
df.head()

,jobs,description,loc,datePosted,workhours,salary,contract_type,experience,diploma,skills,qualifications,industry,company,size,company_description,tools,ID_dep,ville,date
0,ANALYSTE APPLICATIF DATA FINANCE RH (H/F),Dans le cadre d'un important projet avec un gr...,92 - ISSY LES MOULINEAUX,Actualisé le 07 août 2023,35H Travail en journée,50000.0 - 60000.0,Contrat à durée indéterminée,2 ans,NaN,[Adapter les outils de traitement statistique ...,Cadre,Conseil en systèmes et logiciels informatiques,INFORMATIS T.S.,6 à 9 salariés,NaN,NaN,92,ISSY LES MOULINEAUX,2023/08/07
1,Ingénieur DATA (H/F),Nous recherchons : un Ingénieur DATA H/F.\n\nP...,92 - PUTEAUX,Actualisé le 07 août 2023,35H Travail en journée,NaN,Contrat à durée indéterminée,2 ans,"Bac+3, Bac+4 ou équivalents","[Application web, Concevoir une application we...",Agent de maîtrise,Conseil pour les affaires et autres conseils d...,IL CONSULTING,NaN,NaN,"gitlab, kubernetes, mongodb",92,PUTEAUX,2023/08/07
2,Data Integrity Analyst (H/F),ADECCO Lyon Pharma recherche un Data Integrity...,69 - ST PRIEST,Actualisé le 07 août 2023,35H Travail en journée,2311.0,Mission intérimaire - 12 Mois,Débutant accepté,NaN,[Adapter les outils de traitement statistique ...,Employé qualifié,Activités des agences de travail temporaire,ADECCO Onsite,0 salarié (n'ayant pas d'effectif au 31/12 mai...,"Premier réseau d'agences d'emploi en France, A...",NaN,69,ST PRIEST,2023/08/07
3,Technicien Data (h/f),"Adecco PONTARLIER, recherche pour l'un de ses ...",25 - PONTARLIER,Actualisé le 07 août 2023,35H Travail en journée,NaN,Contrat à durée indéterminée,Débutant accepté,NaN,"[Concevoir et gérer un projet, Concevoir un lo...",Employé qualifié,Activités des agences de travail temporaire,ADECCO,6 à 9 salariés,"Premier réseau d'agences d'emploi en France, A...","tableau, power bi",25,PONTARLIER,2023/08/07
4,Data Engineer (H/F),"Selon vos expériences, votre mission sera de :...",33 - BORDEAUX,Actualisé le 07 août 2023,38H30 Travail en journée,30000.0 - 50000.0,Contrat à durée indéterminée,3 ans,NaN,[],Cadre,Conseil en systèmes et logiciels informatiques,THELIO,20 à 49 salariés,NaN,"tableau, power bi, talend, databricks, snowfla...",33,BORDEAUX,2023/08/07


In [1]:
def transform_data(df):
    df = df.copy()
    
    df[['ID_dep', 'ville']] = df['loc'].str.split(' - ', expand = True)
    df['date'] = df['datePosted'].str.replace('Actualisé le ', '').str.replace('Publié le ', '')
    df['date'] = df['date'].apply(lambda x: dateparser.parse(x, languages=['fr']))
    df['date'] = df['date'].dt.strftime('%Y/%m/%d')
    
    df["jobs"] = df["jobs"].str.replace(" h/f", "")
    df["jobs"] = df["jobs"].str.replace("\s*\([hH]/[fF]\)\s*", " ").str.strip()
    df["jobs"] = df["jobs"].str.replace(" (H/F)", "")
    df["jobs"] = df["jobs"].str.replace(" H/F", "")
    df["jobs"] = df["jobs"].str.replace(" F/H", "")
    df["jobs"] = df["jobs"].str.replace(" (F/H)", "")
    df["jobs"] = df["jobs"].str.replace(" \(\)", "")

    
    df = df.applymap(lambda x: ','.join(map(str, x)) if isinstance(x, list) else x)
    df = df.drop(["loc", "datePosted"], axis=1)

    return df

In [2]:
df = transform_data(df)

NameError: name 'df' is not defined

In [ ]:
df.head()

In [ ]:
def count_nan(df):
    
    nan_counts = df.isna().sum() # compte le nombre de NaN pour chaque colonne
    total_counts = len(df) # compte le nombre total de données dans le dataframe
    nan_percentages = (nan_counts / total_counts) * 100 # calcule le pourcentage de NaN pour chaque colonne
    result_df = pd.concat([nan_counts, nan_percentages], axis=1) # combine les deux séries en un dataframe
    result_df.columns = ['NaN Count', 'NaN Percentage'] # renomme les colonnes du nouveau dataframe
    result_df = result_df.sort_values(by = ['NaN Count'], ascending = False)
    return result_df

In [3]:
count_nan(df)

NameError: name 'count_nan' is not defined

### Export Transform df 

In [88]:
def export_silver_to_csv(df):
    # Obtenez la date actuelle
    current_date = date.today().strftime("%d_%m_%Y")

    # Assurez-vous que le dossier de destination existe
    folder_path = f"../data/silver/{current_date}"
    os.makedirs(folder_path, exist_ok=True)

    # Exportez le DataFrame dans un fichier CSV
    csv_file_path = f"{folder_path}/jobs_data.csv"
    df.to_csv(csv_file_path, index=False)

In [89]:
export_silver_to_csv(df)

### Concat silver and transform df

In [90]:
def concat_data(df):
    
    # importer le df qui contient toutes les données déjà récoltées
    df_history = pd.read_csv("../data/silver/silver_data.csv")
    
    # concaténer les données qui viennent d'être récoltées à df_history
    global_df = pd.concat([df_history, df], axis=0)
    sort_df = global_df.sort_values('date', ascending=True)
    
    return sort_df

In [91]:
global_df = concat_data(df)

In [92]:
global_df.head()

,jobs,description,loc,datePosted,workhours,salary,contract_type,experience,diploma,skills,qualifications,industry,company,size,company_description,tools,ID_dep,ville,date
139,Data Engineer sénior (F/H) CDI (H/F),"En tant que Data Engineer chez Quantmetry, vou...",75 - PARIS 08,Actualisé le 12 juillet 2023,38H Travail en journée,40000.0 - 50000.0,Contrat à durée indéterminée,5 ans,NaN,"Concevoir et gérer un projet,Concevoir un logi...",Cadre,Conseil pour les affaires et autres conseils d...,QUANTMETRY,100 à 199 salariés,Pure player en Data et Intelligence Artificiel...,NaN,75,PARIS 08,2023/07/12
138,Data Consultant Stratégie Sénior (H/F),Nous recrutons des personnes avec une appétenc...,75 - PARIS 08,Actualisé le 12 juillet 2023,35H Travail en journée,3100.0 - 5000.0,Contrat à durée indéterminée,5 ans,NaN,"Analyser les résultats d'un projet,Décliner la...",Cadre,Conseil pour les affaires et autres conseils d...,QUANTMETRY,100 à 199 salariés,Pure player en Data et Intelligence Artificiel...,NaN,75,PARIS 08,2023/07/12
138,Data Consultant Stratégie Sénior (H/F),Nous recrutons des personnes avec une appétenc...,75 - PARIS 08,Actualisé le 12 juillet 2023,35H Travail en journée,3100.0 - 5000.0,Contrat à durée indéterminée,5 ans,NaN,"[""Analyser les résultats d'un projet"", ""Déclin...",Cadre,Conseil pour les affaires et autres conseils d...,QUANTMETRY,100 à 199 salariés,Pure player en Data et Intelligence Artificiel...,NaN,75,PARIS 08,2023/07/12
139,Data Engineer sénior (F/H) CDI (H/F),"En tant que Data Engineer chez Quantmetry, vou...",75 - PARIS 08,Actualisé le 12 juillet 2023,38H Travail en journée,40000.0 - 50000.0,Contrat à durée indéterminée,5 ans,NaN,"['Concevoir et gérer un projet', ""Concevoir un...",Cadre,Conseil pour les affaires et autres conseils d...,QUANTMETRY,100 à 199 salariés,Pure player en Data et Intelligence Artificiel...,NaN,75,PARIS 08,2023/07/12
133,Chef de projets Performance Durable/Energie/Da...,Intégré(e) au sein de la Direction Performance...,92 - ASNIERES SUR SEINE,Actualisé le 13 juillet 2023,35H Travail en journée,NaN,Contrat à durée indéterminée,3 ans,NaN,['Contrôler et faire appliquer le respect de d...,Employé qualifié,Activités des sièges sociaux,NEXITY,250 à 499 salariés,Nexity est aujourd hui leader sur les différen...,tableau,92,ASNIERES SUR SEINE,2023/07/13


In [93]:
global_df.shape

(2000, 19)

In [94]:
global_df.isnull().sum()

jobs                      0
description               0
loc                       0
datePosted                0
workhours              1606
salary                 1578
contract_type             0
experience                0
diploma                1870
skills                    0
qualifications          878
industry                844
company                 862
size                    622
company_description     928
tools                   818
ID_dep                    0
ville                   246
date                      0
dtype: int64

In [95]:
loc = global_df.loc[global_df.duplicated()]
loc.shape

(2, 19)

In [143]:
global_df = global_df.drop_duplicates()
global_df.duplicated().sum()

0

In [145]:
global_df["date"].min()

'2023/07/12'

In [144]:
global_df["date"].max()

'2023/08/07'

### Export silver df

In [102]:
def export_silver_df(df):
    csv_file_path = f"../data/silver/silver_data.csv"
    df.to_csv(csv_file_path, index=False)

In [103]:
export_silver_df(df)